In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Baixar stopwords do NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to C:\Users\valdiney.pedro.M
[nltk_data]     ADIBA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


# Carregar base de dados

In [2]:
url = "https://github.com/valdineyatilio/ProjetoAplicado-III/raw/refs/heads/main/Aula-02/BaseDeDados-AmazonProductReviews.csv"
dados_Amazon = pd.read_csv(url, encoding='latin1', sep=';')


# Preencher valores nulos

In [3]:
dados_Amazon['Text'] = dados_Amazon['Text'].fillna('')
dados_Amazon['HelpfulnessNumerator'] = dados_Amazon['HelpfulnessNumerator'].fillna(dados_Amazon['HelpfulnessNumerator'].median())
dados_Amazon['HelpfulnessDenominator'] = dados_Amazon['HelpfulnessDenominator'].fillna(dados_Amazon['HelpfulnessDenominator'].median())


# Aplicar stemming e remoção de stopwords

In [4]:
def preprocess_text(text):
    words = text.lower().split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return " ".join(words)

dados_Amazon['Text'] = dados_Amazon['Text'].apply(preprocess_text)


# Remover colunas irrelevantes

In [5]:
dados_Amazon = dados_Amazon.drop(columns=['UserId', 'ProfileName', 'Summary'])

# Codificar variáveis categóricas

In [6]:
dados_Amazon['ProductId'] = LabelEncoder().fit_transform(dados_Amazon['ProductId'])

# Normalizar variáveis numéricas

In [7]:
scaler = MinMaxScaler()
colunas_para_normalizar = ['HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score']
dados_Amazon[colunas_para_normalizar] = scaler.fit_transform(dados_Amazon[colunas_para_normalizar])


# Ajuste na Transformação de Texto

In [8]:
# Divisão treino/teste
X_train, X_test = train_test_split(dados_Amazon, test_size=0.2, random_state=42)

# Refinar o modelo TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_df=0.85, min_df=3, ngram_range=(1,3))
matriz_tfidf = tfidf.fit_transform(X_train['Text'])

# Calcular similaridade
similaridade = cosine_similarity(matriz_tfidf)

# Ajuste na Função de Recomendação

In [9]:
def recomendar_produto(product_index, top_n=5):
    similares = list(enumerate(similaridade[product_index]))
    similares = sorted(similares, key=lambda x: x[1], reverse=True)

    print(f"\nProdutos semelhantes ao ID {X_train.iloc[product_index]['ProductId']}:\n")
    produtos_recomendados = set()
    
    for i in range(1, top_n + 1):  # Ignorar o primeiro, que é ele mesmo
        produto_id = X_train.iloc[similares[i][0]]['ProductId']
        score = similares[i][1]

        if produto_id not in produtos_recomendados:  # Evitar duplicações
            produtos_recomendados.add(produto_id)
            print(f"- Produto ID: {produto_id}, Similaridade: {score:.4f}")

# Teste de recomendação
produto_escolhido = np.random.randint(0, len(X_train))
recomendar_produto(produto_escolhido)


Produtos semelhantes ao ID 514:

- Produto ID: 322, Similaridade: 0.5438
- Produto ID: 112, Similaridade: 0.0000
- Produto ID: 376, Similaridade: 0.0000


# Avaliação Após os Ajustes

In [10]:
def avaliar_precisao(n_amostras=10):
    precisao_total = 0
    for i in np.random.choice(len(X_train), n_amostras, replace=False):
        similares = list(enumerate(similaridade[i]))
        similares = sorted(similares, key=lambda x: x[1], reverse=True)
        
        score_medio = np.mean([score for _, score in similares[1:6]])  # Média dos top 5 similares
        precisao_total += score_medio

    return precisao_total / n_amostras

def avaliar_cobertura():
    produtos_recomendados = set()
    for i in np.random.choice(len(X_train), 100, replace=False):
        similares = list(enumerate(similaridade[i]))
        similares = sorted(similares, key=lambda x: x[1], reverse=True)
        produtos_recomendados.update([X_train.iloc[idx]['ProductId'] for idx, _ in similares[1:6]])

    return len(produtos_recomendados) / len(X_train['ProductId'].unique())

def avaliar_diversidade():
    diversidade_total = 0
    for i in np.random.choice(len(X_train), 10, replace=False):
        similares = list(enumerate(similaridade[i]))
        similares = sorted(similares, key=lambda x: x[1], reverse=True)
        categorias = [X_train.iloc[idx]['Text'][:20] for idx, _ in similares[1:6]]  # Trecho do texto para diversidade
        diversidade_total += len(set(categorias)) / len(categorias)

    return diversidade_total / 10

# Teste das métricas
print(f"\nPrecisão média das recomendações: {avaliar_precisao():.4f}")
print(f"Cobertura dos produtos recomendados: {avaliar_cobertura():.4f}")
print(f"Diversidade das recomendações: {avaliar_diversidade():.4f}")


Precisão média das recomendações: 0.4024
Cobertura dos produtos recomendados: 0.2721
Diversidade das recomendações: 0.9600
